# WebSurferAgent

AutoGen provides a proof-of-concept WebSurferAgent that can command a simple text-based browser (similar to [Lynx](https://en.wikipedia.org/wiki/Lynx_(web_browser))) to search the web, visit pages, navigate within pages, download files, etc. The browsing is stateful, meaning that browsing history, viewport state, and other details are maintained throughout the conversation. 

This work was largely inspired by OpenAI's [WebGPT](https://openai.com/research/webgpt) project from December 2021. 

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install AutoGen with the optional `websurfer` dependencies:
```bash
pip install "autogen-agentchat[websurfer]~=0.2"
```

In [1]:
# %pip install --quiet "autogen-agentchat[websurfer]~=0.2"

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well).

The WebSurferAgent uses a combination of models. GPT-4 and GPT-3.5-turbo-16 are recommended.

Your json config should look something like the following:
```json
[
    {
        "model": "gpt-4",
        "api_key": "<your OpenAI API key here>"
    },
    {
        "model": "gpt-3.5-turbo-16k",
        "api_key": "<your OpenAI API key here>"
    }
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choose "upload file" icon.


In [2]:
import autogen  # noqa: E402

llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={"model": ["gpt-4", "gpt-4-0613", "gpt-4-32k", "gpt-4-32k-0613", "gpt-4-1106-preview"]},
    ),
    "temperature": 0,
}

summarizer_llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={"model": ["gpt-3.5-turbo-1106", "gpt-3.5-turbo-16k-0613", "gpt-3.5-turbo-16k"]},
    ),
    "temperature": 0,
}

## Configure Bing

For WebSurferAgent to be reasonably useful, it needs to be able to search the web -- and that means it needs a Bing API key. 
You can read more about how to get an API on the [Bing Web Search API](https://www.microsoft.com/en-us/bing/apis/bing-web-search-api) page.

Once you have your key, either set it as the `BING_API_KEY` system environment variable, or simply input your key below.


In [3]:
import os  # noqa: E402

bing_api_key = os.environ["BING_API_KEY"]

### Construct Agents

We now create out WebSurferAgent, and a UserProxyAgent to surf the web. 

In [4]:
from autogen.agentchat.contrib.web_surfer import WebSurferAgent  # noqa: E402
from autogen.browser_utils import BingMarkdownSearch, RequestsMarkdownBrowser  # noqa: E402

browser = RequestsMarkdownBrowser(
    downloads_folder=os.getcwd(),
    search_engine=BingMarkdownSearch(bing_api_key=bing_api_key),
)

web_surfer = WebSurferAgent(
    "web_surfer",
    llm_config=llm_config,
    summarizer_llm_config=summarizer_llm_config,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    browser=browser,
)

user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    code_execution_config=False,
    default_auto_reply="",
    is_termination_msg=lambda x: True,
)

### Example 1: Search, summarize
- Search for information about Microsoft AutoGen
- Summarize the results
- Visit the Getting Started Docs page

In [5]:
task1 = """
Search the web for information about Microsoft AutoGen
"""

user_proxy.initiate_chat(web_surfer, message=task1);

user_proxy (to web_surfer):


Search the web for information about Microsoft AutoGen


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION informational_web_search...
web_surfer (to user_proxy):

Address: search: Microsoft AutoGen
Title: Microsoft AutoGen - Search
Viewport position: Showing page 1 of 1.
## A Bing search for 'Microsoft AutoGen' found 19 results:

1. [AutoGen: Enabling next-generation large language model applications](https://www.microsoft.com/en-us/research/blog/autogen-enabling-next-generation-large-language-model-applications/)
AutoGen enables complex LLM-based workflows using multi-agent conversations. (Left) AutoGen agents are customizable and can be based on LLMs, tools, humans, and even a combination of them. (Top-right) Agents can converse to solve tasks. (Bottom-right) The framework supports many additional complex conversation patterns.

2. [AutoGen - Microsoft Research](ht

In [6]:
task2 = "Summarize these results"
user_proxy.initiate_chat(web_surfer, message=task2, clear_history=False);

user_proxy (to web_surfer):

Summarize these results

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION summarize_page...
web_surfer (to user_proxy):

Microsoft AutoGen is a framework that enables the development of large language model (LLM) applications using multiple agents that can converse with each other to solve tasks. It allows for the creation of diverse teams of agents with specialized skills or goals, facilitating greater diversity in workflows. AutoGen agents are customizable, conversable, and seamlessly allow human participation. The framework supports various conversation patterns and offers a collection of working systems spanning a wide range of applications. AutoGen Studio is an interface powered by AutoGen that allows for the rapid prototyping of multi-agent solutions through a point and click, drag and drop interface. There are also tutorials and videos available to help users g

In [7]:
task3 = "Click the 'Getting Started' result"
user_proxy.initiate_chat(web_surfer, message=task3, clear_history=False);

user_proxy (to web_surfer):

Click the 'Getting Started' result

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION visit_page...
web_surfer (to user_proxy):

Address: https://microsoft.github.io/autogen/docs/Getting-Started/
Title: Getting Started | AutoGen
Viewport position: Showing page 1 of 1.

[Skip to main content](#__docusaurus_skipToContent_fallback)What's new in AutoGen? Read [this blog](/autogen/blog/2024/03/03/AutoGen-Update) for an overview of updates[![AutoGen](/autogen/img/ag.svg)![AutoGen](/autogen/img/ag.svg)**AutoGen**](/autogen/)Docs* [Getting Started](/autogen/docs/Getting-Started)
* [Installation](/autogen/docs/installation/)
* [Tutorial](/autogen/docs/tutorial/introduction)
* [User Guide](/autogen/docs/topics)
* [API Reference](/autogen/docs/reference/agentchat/conversable_agent)
* [FAQs](/autogen/docs/FAQ)
* [Ecosystem](/autogen/docs/ecosystem)
* [Contribute](/autogen/docs/Con

### Example 2: Navigational search, scroll, answer questions
- Search for Microsoft's wikipedia page, then naviagate to it
- Scroll down
- Answer questions about the content

In [8]:
task4 = """Find Microsoft's Wikipedia page."""
user_proxy.initiate_chat(web_surfer, message=task4, clear_history=False);

user_proxy (to web_surfer):

Find Microsoft's Wikipedia page.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION navigational_web_search...
web_surfer (to user_proxy):

Address: https://en.wikipedia.org/wiki/Microsoft
Title: Microsoft
Viewport position: Showing page 1 of 34.
# Microsoft

American multinational technology corporation

Microsoft Corporation
| [A square divided into four sub-squares, colored red-orange, green, yellow and blue (clockwise), with the company name appearing to its right](/wiki/File%3AMicrosoft_logo_%282012%29.svg) | |
| --- | --- |
| Aerial view of the [Microsoft Redmond campus](/wiki/Microsoft_Redmond_campus "Microsoft Redmond campus") | |
| Company type | [Public](/wiki/Public_company "Public company") |
| [Traded as](/wiki/Ticker_symbol "Ticker symbol") | * [Nasdaq](/wiki/Nasdaq "Nasdaq"): [MSFT](https://www.nasdaq.com/market-activity/stocks/msft) * [Nasdaq-100](/wiki

In [9]:
task5 = """Scroll down."""
user_proxy.initiate_chat(web_surfer, message=task5, clear_history=False);

user_proxy (to web_surfer):

Scroll down.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION page_down...
web_surfer (to user_proxy):

Address: https://en.wikipedia.org/wiki/Microsoft
Title: Microsoft
Viewport position: Showing page 2 of 34.
Gates series") |

**Microsoft Corporation** is an American [multinational corporation](/wiki/Multinational_corporation "Multinational corporation") and [technology company](/wiki/Technology_company "Technology company") headquartered in [Redmond, Washington](/wiki/Redmond%2C_Washington "Redmond, Washington").[[2]](#cite_note-2) Microsoft's best-known [software products](/wiki/List_of_Microsoft_software "List of Microsoft software") are the [Windows](/wiki/Microsoft_Windows "Microsoft Windows") line of [operating systems](/wiki/List_of_Microsoft_operating_systems "List of Microsoft operating systems"), the [Microsoft 365](/wiki/Microsoft_365 "Microsoft 365") su

In [10]:
task6 = """Read the page and answer: Where was the first office location, and when did they move to Redmond?"""
user_proxy.initiate_chat(web_surfer, message=task6, clear_history=False);

user_proxy (to web_surfer):

Read the page and answer: Where was the first office location, and when did they move to Redmond?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION read_page_and_answer...
web_surfer (to user_proxy):

Microsoft Corporation, an American multinational technology company, was founded on April 4, 1975, in Albuquerque, New Mexico, by Bill Gates and Paul Allen. The company's first office location was in Albuquerque, but they later moved their headquarters to Redmond, Washington. The move to Redmond occurred in January 1979. Since then, Microsoft has become a major player in the technology industry, developing and selling software products such as the Windows operating system, Microsoft Office suite, and Xbox video game consoles. They have also expanded into cloud computing with Microsoft Azure and have made acquisitions such as Nokia's mobile unit and LinkedIn.

-----------